In [1]:
import pandas as pd
import numpy as np
from scipy import stats

## Import file

In [2]:
bank_df = pd.read_excel("bank_deposit.xlsx")
bank_df.head()
# bank_df.shape # 11,222 rows, columns 17

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,207.0,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,215.0,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,212.0,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,860.0,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,476.0,2.0,13.0,9.0,success,yes


## Assessement of anomalies

In [3]:
bank_df.isna().sum()
# education - replace with unknown as is an option in the categories
# 6 NA categories all in same rows (housing, default, balance,contact)

age                      0
job                      0
marital                  6
education               27
default                  6
balance                  6
housing                  6
loan                    12
contact                  6
day                      6
month                    8
duration                 8
campaign                 8
pdays                    8
previous                 6
poutcome                 0
Bank deposit(target)     0
dtype: int64

In [4]:
# To drop the rows that contained NAN in the contact column, as cannot be contacted and thus useless data
#bank_df= bank_df[bank_df['contact'].isna()]
#bank_df

In [5]:
bank_df[bank_df.isna().any(axis=1)]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
21,86,retired,divorced,NaN,no,0.0,no,no,telephone,26.0,aug,189.0,1.0,-1.0,0.0,unknown,yes
22,86,retired,divorced,NaN,no,157.0,no,no,telephone,7.0,sep,147.0,1.0,-1.0,0.0,unknown,no
23,86,retired,single,NaN,no,614.0,no,no,telephone,9.0,dec,595.0,1.0,-1.0,0.0,unknown,yes
24,86,retired,married,NaN,no,1817.0,no,no,cellular,16.0,mar,272.0,3.0,-1.0,0.0,unknown,no
25,86,retired,married,NaN,no,1503.0,no,no,telephone,18.0,mar,165.0,3.0,101.0,1.0,other,no
26,86,retired,married,NaN,no,5236.0,no,no,telephone,1.0,apr,558.0,2.0,-1.0,0.0,unknown,yes
27,86,retired,married,NaN,no,1255.0,no,no,telephone,14.0,jul,247.0,1.0,180.0,3.0,success,yes
28,86,retired,married,NaN,no,1255.0,no,no,cellular,14.0,oct,192.0,2.0,92.0,4.0,success,no
29,85,retired,married,NaN,no,12114.0,no,no,telephone,12.0,feb,165.0,2.0,-1.0,0.0,unknown,yes
30,85,retired,divorced,NaN,no,7613.0,no,no,cellular,18.0,feb,355.0,3.0,-1.0,0.0,unknown,yes


In [6]:
bank_df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,11222.000000,1.121600e+04,11216.000000,1.121400e+04,11214.000000,11214.000000,11216.000000
mean,56.411068,7.966974e+03,15.786912,2.040672e+03,2.737739,35.118245,0.558934
std,6.141462,6.421456e+05,8.336913,1.888620e+05,2.854410,90.776604,1.741345
min,50.000000,-4.057000e+03,1.000000,0.000000e+00,1.000000,-1.000000,0.000000
25%,52.000000,1.080000e+02,8.000000,1.020000e+02,1.000000,-1.000000,0.000000
50%,55.000000,6.275000e+02,16.000000,1.760000e+02,2.000000,-1.000000,0.000000
75%,58.000000,2.031750e+03,21.000000,3.160000e+02,3.000000,-1.000000,0.000000
max,95.000000,6.800000e+07,31.000000,2.000000e+07,43.000000,792.000000,37.000000


In [7]:
# Droping customer index 38 because it is a potential duplicate with corrupted "duration" value.
## this can be done with the Remove_outlier function at the bottom thus commented out
# bank_df = bank_df.drop(index=38)

In [8]:
# Dropped the rows that contained NAN in the contact column.
bank_df= bank_df[bank_df['contact'].notna()]
bank_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,207.0,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,215.0,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,212.0,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,860.0,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,476.0,2.0,13.0,9.0,success,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,50,entrepreneur,divorced,tertiary,no,315.0,no,no,cellular,12.0,jul,101.0,2.0,185.0,2.0,failure,no
11218,50,management,married,tertiary,no,2881.0,no,no,cellular,5.0,aug,510.0,2.0,2.0,5.0,other,no
11219,50,technician,divorced,secondary,no,0.0,yes,no,cellular,13.0,aug,120.0,2.0,91.0,6.0,failure,no
11220,50,services,married,secondary,no,2376.0,yes,no,cellular,24.0,sep,395.0,1.0,-1.0,0.0,unknown,yes


In [9]:
#Checking for duplicate rows
bank_df[bank_df.duplicated(keep=False)].sort_values(by=['balance'], ascending = True)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
3713,57,self-employed,married,tertiary,yes,-3313.0,yes,yes,unknown,9.0,may,153.0,1.0,-1.0,0.0,unknown,no
3753,57,self-employed,married,tertiary,yes,-3313.0,yes,yes,unknown,9.0,may,153.0,1.0,-1.0,0.0,unknown,no
9209,51,entrepreneur,married,secondary,yes,-2082.0,no,yes,cellular,28.0,jul,123.0,6.0,-1.0,0.0,unknown,no
9647,51,entrepreneur,married,secondary,yes,-2082.0,no,yes,cellular,28.0,jul,123.0,6.0,-1.0,0.0,unknown,no
8203,52,management,married,tertiary,no,-1224.0,yes,no,cellular,7.0,may,253.0,4.0,-1.0,0.0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,75,retired,married,secondary,no,26452.0,no,no,telephone,15.0,jul,219.0,2.0,-1.0,0.0,unknown,no
3944,57,technician,married,tertiary,no,27069.0,no,yes,unknown,20.0,jun,174.0,3.0,-1.0,0.0,unknown,no
3697,57,technician,married,tertiary,no,27069.0,no,yes,unknown,20.0,jun,174.0,3.0,-1.0,0.0,unknown,no
1356,60,retired,married,primary,no,71188.0,no,no,cellular,6.0,oct,205.0,1.0,-1.0,0.0,unknown,no


In [10]:
# ASSESS OUTLIERS USING Z SCORE of 3
    # as 98% of data within 3 SD of the mean, checkin what ouliers there are
    # BALANCE column
bank_df[np.abs(stats.zscore(bank_df['balance']) > 3)] 
    ## used for visually checking data
    # one outlier identifed and should be removed


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
231,77,retired,married,primary,no,68000000.0,no,no,telephone,27.0,nov,341.0,4.0,94.0,3.0,failure,no


In [11]:
bank_df['duration']=pd.DataFrame(bank_df['duration'].apply(lambda x: x/60))

bank_df[np.abs(stats.zscore(bank_df['duration']) > 3)]
# one outlier and should be removed

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)


In [12]:
# Outliers for number of campaign calls
bank_df[np.abs(stats.zscore(bank_df['campaign']) > 3)] 
    # no specific outlier identified as needing processing, but poses questions for how many times a client should be called
    # thus no action to be taken with regards to cleaning

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)


In [13]:
# Outliers for pdays - numbers of days since last contact in previous campaign
bank_df[(np.abs(stats.zscore(bank_df['pdays']) > 3))]
    # dont seem unusual but could pose a question for later, i.e. should these clients be contacted as priority
    # 345 rows/clients

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)


In [14]:
# Outliers for previous - numbers of contacts before this campaign
bank_df[(np.abs(stats.zscore(bank_df['previous']) > 3))]
    # dont seem unusual but could pose a question for later, i.e. what is the number of contacts the favor a succesful outcome

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,14.333333,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,7.933333,2.0,13.0,9.0,success,yes
68,83,housemaid,married,primary,no,2140.0,no,no,cellular,6.0,sep,1.816667,3.0,276.0,8.0,other,no
108,80,retired,married,secondary,no,8304.0,no,no,telephone,9.0,jun,11.866667,1.0,64.0,12.0,failure,yes
109,80,retired,married,secondary,no,8304.0,no,no,telephone,6.0,apr,11.350000,1.0,118.0,11.0,success,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11194,50,management,married,tertiary,no,78.0,yes,no,cellular,14.0,jan,3.633333,3.0,161.0,9.0,other,no
11204,50,technician,married,tertiary,no,0.0,no,yes,cellular,12.0,apr,5.616667,1.0,258.0,6.0,failure,no
11215,50,technician,married,tertiary,no,0.0,no,yes,cellular,6.0,jul,4.283333,1.0,85.0,7.0,failure,no
11216,50,unknown,married,unknown,no,558.0,no,no,cellular,9.0,jul,7.733333,1.0,683.0,6.0,failure,yes


In [15]:
# checking that days of the month coherent, 
bank_df['day'].max()
bank_df['day'].min()
    # between 1 and 31 which is coherent, thus no further action required

1.0

## Implementation of data cleaning decisions

In [16]:
#Changed NAN into unknown in the "education" column.
bank_df["education"] = bank_df["education"].fillna("unknown")

In [17]:
# Drop all duplicates in the DataFrame
bank_df = bank_df.drop_duplicates()
bank_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,3.450000,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,3.583333,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,3.533333,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,14.333333,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,7.933333,2.0,13.0,9.0,success,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11215,50,technician,married,tertiary,no,0.0,no,yes,cellular,6.0,jul,4.283333,1.0,85.0,7.0,failure,no
11217,50,entrepreneur,divorced,tertiary,no,315.0,no,no,cellular,12.0,jul,1.683333,2.0,185.0,2.0,failure,no
11219,50,technician,divorced,secondary,no,0.0,yes,no,cellular,13.0,aug,2.000000,2.0,91.0,6.0,failure,no
11220,50,services,married,secondary,no,2376.0,yes,no,cellular,24.0,sep,6.583333,1.0,-1.0,0.0,unknown,yes


In [18]:
def replace_w_median(bank_df):
    # find median for columns duration, campaign, pdays, and replace with mode for month
    bank_df['duration'].fillna(bank_df['duration'].median(), inplace=True)
    bank_df['campaign'].fillna(bank_df['campaign'].median(), inplace=True)
    bank_df['pdays'].fillna(bank_df['pdays'].median(), inplace=True)
    bank_df['month'].fillna(bank_df['month'].mode(),inplace=True)
replace_w_median(bank_df)

C:\Users\berka\AppData\Local\Temp\ipykernel_23928\2059589770.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df['duration'].fillna(bank_df['duration'].median(), inplace=True)
C:\Users\berka\AppData\Local\Temp\ipykernel_23928\2059589770.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df['campaign'].fillna(bank_df['campaign'].median(), inplace=True)
C:\Users\berka\AppData\Local\Temp\ipykernel_23928\2059589770.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [19]:
# quantiles for the pivot table
def quantile_25(bank_df:pd.Series):
    return bank_df.quantile(.25)

def quantile_75(bank_df:pd.Series):
    return bank_df.quantile(.75)

In [20]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,3.450000,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,3.583333,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,3.533333,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,14.333333,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,7.933333,2.0,13.0,9.0,success,yes


In [21]:
bank_pivot=bank_df.pivot_table(values='balance', index=['Bank deposit(target)','loan'], aggfunc=[ quantile_25, np.median, quantile_75,np.max])
bank_pivot

quantile_25  median quantile_75        amax
                              balance balance     balance     balance
Bank deposit(target) loan                                            
no                   no         113.0   647.0      2026.0  68000000.0
                     yes         14.0   314.0      1121.5     51439.0
yes                  no         351.0  1230.0      3025.0     52587.0
                     yes        119.0   448.0      1783.0      9103.0

In [22]:
#Median1 is the value for Bank deposit=no, loan=no
median1=int(bank_pivot.loc[(["no"],["no"]),(["median"],["balance"])].values[0])
#Median2 is the value for Bank deposit=yes, loan=no
median2=int(bank_pivot.loc[(["yes"],["no"]),(["median"],["balance"])].values[0])

#These filters allows us to check for multiple conditions and change the "loan" column according to these conditions with a .loc method afterwards.
filter1 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="no") & (bank_df['balance'] > median1)
bank_df.loc[filter1, "loan"] = "no"
filter2 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="no") & (bank_df['balance'] < median1)
bank_df.loc[filter2, "loan"] = "yes"
filter3 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="yes") & (bank_df['balance'] > median2)
bank_df.loc[filter3, "loan"] = "no"
filter4 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="yes") & (bank_df['balance'] < median2)
bank_df.loc[filter4, "loan"] = "yes"

In [23]:

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

bank_df.head(100)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,3.450000,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,3.583333,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,3.533333,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,14.333333,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,7.933333,2.0,13.0,9.0,success,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,81,retired,married,primary,no,5220.0,no,no,telephone,13.0,nov,2.250000,1.0,94.0,2.0,other,yes
101,81,retired,married,secondary,no,93.0,no,no,cellular,7.0,dec,2.733333,1.0,-1.0,0.0,unknown,no
102,81,retired,divorced,secondary,no,452.0,no,no,telephone,16.0,dec,7.383333,1.0,-1.0,0.0,unknown,yes
103,81,retired,divorced,tertiary,no,0.0,no,no,cellular,14.0,jan,5.950000,1.0,-1.0,0.0,unknown,no


In [24]:
#Fixed weird cells in education and contact
bank_df.loc[bank_df["education"]=="hjkl", "education"] = "unknown"
bank_df.loc[bank_df["contact"]=="ghjk", "contact"] = "telephone"

In [25]:
## we can take this out
#bank_df['outlier_bal']= np.where(bank_df['balance']> (bank_df['balance'].mean()+(bank_df['balance'].std()*3)), True,False)
#bank_df=bank_df.drop(bank_df[bank_df['outlier_bal']==True].index)

In [26]:
def remove_outliers(bank_df):
    
    bank_df.drop(bank_df[np.abs(stats.zscore(bank_df['balance']) > 3)==True].index, inplace=True)
    bank_df.drop(bank_df[np.abs(stats.zscore(bank_df['duration']) > 3)==True].index,inplace=True)
    

remove_outliers(bank_df)


C:\Users\berka\AppData\Local\Temp\ipykernel_23928\2199978985.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df.drop(bank_df[np.abs(stats.zscore(bank_df['balance']) > 3)==True].index, inplace=True)
C:\Users\berka\AppData\Local\Temp\ipykernel_23928\2199978985.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df.drop(bank_df[np.abs(stats.zscore(bank_df['duration']) > 3)==True].index,inplace=True)


In [27]:
#Data type correction from float64 to int64.
list_of_floats_to_integer = ['day', 'duration', 'campaign', 'pdays', 'previous']

for i in list_of_floats_to_integer:
    bank_df[i] = bank_df[i].astype(np.int64)

C:\Users\berka\AppData\Local\Temp\ipykernel_23928\4198514806.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df[i] = bank_df[i].astype(np.int64)


In [28]:
bank_df.to_csv('csv_export4.csv')

In [29]:
bank_df[bank_df['previous'].isna()]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)


In [30]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21,apr,3,17,-1,0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1,oct,3,1,-1,0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3,mar,3,1,-1,0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22,jul,14,2,177,7,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4,aug,7,2,13,9,success,yes
